### Imports

In [300]:
import os
SET_SEED=42
os.environ['PYTHONHASHSEED'] = str(SET_SEED)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [301]:
import torch
import copy
import sklearn
import gc
import timm
import time
import safetensors
import random
import warnings
import torch.nn as nn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from torchvision.models import resnet50, ResNet50_Weights
from safetensors.torch import load_file
from PIL import Image
from torch.utils.data import DataLoader, random_split
from sklearn.preprocessing import StandardScaler
from torchvision import transforms, models
from collections import defaultdict
from sklearn.model_selection import GroupKFold
from scipy.stats import zscore, norm, laplace

warnings.filterwarnings("ignore", message=".*does not have a deterministic implementation.*")

# if os.environ.get('KAGGLE_KERNEL_RUN_TYPE') != 'Batch':
#     !pip install -q ipdb
#     import ipdb

In [302]:
def set_seed(seed=SET_SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except AttributeError:
        pass  # Older PyTorch versions
    

set_seed(SET_SEED)

In [303]:
# Hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16
NUM_FT_EPOCHS = 20
NUM_BB_EPOCHS = 12
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 1e-4
NUM_FOLDS = 1
GIVEN_WEIGHTS = [0.1, 0.1, 0.1, 0.5, 0.2]
TARGET_COLS = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']
BINARY_COLS = TARGET_COLS[:3]
BASE_MODEL='efficientnet_b1'
IMAGE_SIZE=(392,392)
TRAIN_SHUFFLE=0

In [322]:
def print_result(train_loss, val_loss, epoch_start, epoch, num_epochs, val_r2):
    epoch_time = time.time() - epoch_start
    mins, secs = divmod(epoch_time, 60)
    
    print(f'Epoch {epoch+1}/{num_epochs} - '
          f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f} | '
          f'R²: {val_r2:.4f} | '
          f'Time: {int(mins)}m {int(secs)}s')

class BiomassDataset(torch.utils.data.Dataset):
    def __init__(self, df, base_path, transform=None):
        self.df = df
        self.base_path = base_path
        self.transform = transform
        self.target_cols = TARGET_COLS
        self.is_training = all(col in df.columns for col in self.target_cols)
        if self.is_training:
            self.is_nonzero = (df[BINARY_COLS].values > 0).astype(np.float32)
            self.targets = df[self.target_cols].values.astype(np.float32)
            self.targets_log = np.log1p(np.where(self.targets > 0, self.targets, 0))
            self.targets_mean = np.nanmean(self.targets_log, axis=0)
            self.targets_std = np.nanstd(self.targets_log, axis=0) + 1e-8
            self.targets_norm = (self.targets_log - self.targets_mean) / self.targets_std
            self.targets_norm_masked = self.targets_norm.copy()
            self.targets_norm_masked[:, :3] = np.where(
                self.is_nonzero, self.targets_norm[:, :3], 0.0
            ) # first 3 cols
            print(self.targets_norm_masked) 

    def denormalize(self, normalized_targets):
        device = normalized_targets.device
        means = torch.tensor(self.targets_mean, dtype=torch.float32, device=device)
        stds = torch.tensor(self.targets_std, dtype=torch.float32, device=device)
        if normalized_targets.dim() == 1:
            return normalized_targets * stds + means
        else:
            return normalized_targets * stds.unsqueeze(0) + means.unsqueeze(0)
    
    def __len__(self):
        return len(self.df)

    def _get_crop(self, image, crop_type):
        """Get different crop from image"""
        width, height = image.size
        
        if crop_type == 0:  # Left half
            return image.crop((0, 0, width // 2, height))
        elif crop_type == 1:  # Right half
            return image.crop((width // 2, 0, width, height))
        elif crop_type == 2:  # Top half
            return image.crop((0, 0, width, height // 2))
        elif crop_type == 3:  # Bottom half
            return image.crop((0, height // 2, width, height))
        else:  # Center crop (80% of image)
            crop_w, crop_h = int(width * 0.8), int(height * 0.8)
            left = (width - crop_w) // 2
            top = (height - crop_h) // 2
            return image.crop((left, top, left + crop_w, top + crop_h))
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx // 5] 
        img_path = os.path.join(self.base_path, row['image_path'])
        
        image = Image.open(img_path).convert('RGB')
        # crop_type = idx % 5
        # image = self._get_crop(image, crop_type)
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_training:
            is_nonzero = torch.tensor(self.is_nonzero[idx, :3], dtype=torch.float32)
            targets = torch.tensor(self.targets_norm_masked[idx], dtype=torch.float32)
            return image, (is_nonzero, targets)
            # targets = row[self.target_cols].values.astype('float32')
            # targets_normalized = (targets - TARGET_MEANS.numpy()) / TARGET_STDS.numpy()
            
            # return image, torch.tensor(targets_normalized, dtype=torch.float32)
        else:            
            return image, row['image_path']

In [305]:
class FinetuneModel(nn.Module):
    def __init__(self, pretrained_backbone=None):
        super().__init__()
        self.backbone = timm.create_model('tf_efficientnet_b1', pretrained=False, num_classes=0)
        model_path = '/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b1/1/tf_efficientnet_b1_aa-ea7a6ee0.pth'
        checkpoint = torch.load(model_path)
        self.backbone.load_state_dict(checkpoint, strict=False)
        
        feature_dim = self.backbone.num_features

        self.binary_head = nn.Sequential(
            nn.Linear(feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 3),
            nn.Sigmoid()  # output probability for each target
        )

        self.regression_head = nn.Sequential(
            nn.Linear(feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(64, 5), # 5 outputs for competition
            # nn.ReLU()
        )

        self._init_head_weights()
    
    def _init_head_weights(self):
        """Initialize regression head with deterministic weights"""
        for m in self.regression_head.modules():
            if isinstance(m, nn.Linear):
                # Use a fixed seed for weight initialization
                with torch.random.fork_rng():
                    torch.manual_seed(SET_SEED)
                    torch.nn.init.xavier_uniform_(m.weight)
                    if m.bias is not None:
                        torch.nn.init.zeros_(m.bias)
    
    def forward(self, x):
        features = self.backbone(x)
        binary_out = self.binary_head(features) 
        regression_out = self.regression_head(features) 
        return binary_out, regression_out

In [306]:
base = '/kaggle/input'
train_csv = f'{base}/csiro-biomass/train.csv'
test_csv = f'{base}/csiro-biomass/test.csv'
extra_csv = f'{base}/grassclover-dataset/biomass_data/train/biomass_train_data.csv'
extra_img = f'{base}/grassclover-dataset/biomass_data/train/images'
base_path = f'{base}/csiro-biomass/'
submission_path = f'{base}/csiro-biomass/sample_submission.csv'

dataset_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)
extra_df = pd.read_csv(extra_csv, sep=';')
extra_img_path = extra_img
unique_test_images = test_df['image_path'].unique()

In [307]:
dataset_df['Sampling_Date'] = pd.to_datetime(dataset_df['Sampling_Date'], format='mixed')  # adjust format if needed
dataset_df = dataset_df.pivot(
    index=['image_path','Sampling_Date'],
    columns='target_name',
    values='target'
).reset_index()
dataset_df['Month'] = dataset_df['Sampling_Date'].dt.month
dataset_df.columns

Index(['image_path', 'Sampling_Date', 'Dry_Clover_g', 'Dry_Dead_g',
       'Dry_Green_g', 'Dry_Total_g', 'GDM_g', 'Month'],
      dtype='object', name='target_name')

### Pytorch

In [310]:
train_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),  # Grass can be flipped
    transforms.RandomRotation(degrees=15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ColorJitter(
        brightness=0.1,  # Sunlight variations
        contrast=0.1,     # Different lighting
        saturation=0.1,  # Grass color variations
        hue=0.1
    ),
    transforms.RandomApply([ 
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.5)) # Advanced augmentations
    ], p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

### Support Functions

In [311]:
def forward_pass(images, is_nonzero, targets, model, optimizer=None):
    images = images.to(device)
    is_nonzero = is_nonzero.to(device)
    targets = targets.to(device)

    pred_binary, pred_regression = model(images)

    # Only first 3 columns for BCE
    loss = combined_hurdle_loss(pred_binary, pred_regression, is_nonzero, targets)

    if optimizer is not None:  # training
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return loss, pred_binary, pred_regression

### Train Model

In [312]:
def combined_hurdle_loss(pred_binary, pred_regression, is_nonzero, targets, given_weights=GIVEN_WEIGHTS):
    device = pred_regression.device
    weights_full = torch.tensor(given_weights, device=device)

    # -------------------------
    # 1️⃣ Binary BCE (first 3 targets)
    # -------------------------
    bce_loss_fn = nn.BCELoss()
    loss_binary = bce_loss_fn(pred_binary, is_nonzero)

    # -------------------------
    # 2️⃣ Regression loss (all 5 targets)
    # -------------------------
    # Create full mask: first 3 columns use is_nonzero, last 2 columns = 1
    mask_full = torch.ones_like(pred_regression, device=device)
    mask_full[:, :3] = is_nonzero  # only mask zeros for first 3 columns

    # Smooth L1 + MSE
    smooth_l1 = nn.SmoothL1Loss(reduction='none')
    mse = nn.MSELoss(reduction='none')

    smooth_l1_loss = smooth_l1(pred_regression, targets)
    mse_loss = mse(pred_regression, targets)

    combined_loss = 0.3 * smooth_l1_loss + 0.7 * mse_loss

    # Apply mask and weights
    weighted_loss = (combined_loss * weights_full * mask_full).sum() / mask_full.sum()

    return loss_binary + weighted_loss

In [313]:

def weighted_r2_score(sum_target, valid_samples, sum_target_sq, ss_res):
    """
    Corrected weighted R² score calculation.
    
    Args:
        sum_target: Sum of targets per column [5]
        valid_samples: Number of valid samples per column [5] (accounts for masking)
        sum_target_sq: Sum of squared targets per column [5]
        ss_res: Sum of squared residuals per column [5]
    
    Returns:
        Weighted R² score
    """
    # Compute R² per column
    r2_per_output = torch.zeros(5, device=device)
    for col in range(5):
        if valid_samples[col] > 0:
            mean_target = sum_target[col] / valid_samples[col]
            ss_tot = sum_target_sq[col] - valid_samples[col] * (mean_target ** 2)
            r2_per_output[col] = 1 - ss_res[col] / (ss_tot + 1e-10)
    
    # Weight the R² scores
    weights = torch.tensor(GIVEN_WEIGHTS, device=device)
    r2_weighted = (r2_per_output * weights).sum() / weights.sum()
    return r2_weighted

In [318]:
def create_data_loaders(train_df, val_df, batch_size=BATCH_SIZE):
    train_dataset = BiomassDataset(train_df, base_path, transform=train_transform)
    val_dataset = BiomassDataset(val_df, base_path, transform=val_transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=TRAIN_SHUFFLE, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    return train_loader, val_loader, train_dataset, val_dataset
    
def train_epoch(model, train_loader, optimizer):
    model.train()
    total_loss = 0

    for images, (is_nonzero, targets) in train_loader:
        loss, _, _ = forward_pass(images, is_nonzero, targets, model, optimizer)
        total_loss += loss.item()

    return total_loss / len(train_loader)

def validate_epoch(model, val_loader, val_dataset):
    """
    Corrected validation function with proper R² calculation.
    
    Key fixes:
    - After denormalization, corrects masked zeros (sets to 0 instead of mean)
    - Converts from log1p back to original scale using expm1
    - Properly masks zeros for first 3 columns in R² calculation
    - Computes R² per column with correct sample counts
    """
    model.eval()
    total_loss = 0

    ss_res = torch.zeros(5, device=device)
    sum_target = torch.zeros(5, device=device)
    sum_target_sq = torch.zeros(5, device=device)
    # Track valid samples per column (for first 3 columns, only count non-zero targets)
    valid_samples = torch.zeros(5, device=device)

    with torch.no_grad():
        for images, (is_nonzero, targets) in val_loader:
            images = images.to(device)
            is_nonzero = is_nonzero.to(device)
            targets = targets.to(device)
            loss, pred_binary, pred_regression = forward_pass(
                images, is_nonzero, targets, model, optimizer=None
            )
            total_loss += loss.item()
    
            # Denormalize: this gives us log1p values
            pred_log = val_dataset.denormalize(pred_regression)
            targets_log = val_dataset.denormalize(targets)
            
            # FIX: For first 3 columns, if is_nonzero=0, the normalized value was 0.0
            # Denormalizing 0.0 gives mean_log1p, but we want log1p(0) = 0
            # So set masked zeros to 0
            targets_log[:, :3] = targets_log[:, :3] * is_nonzero
            
            # Convert from log1p back to original scale using expm1
            pred_original = torch.expm1(pred_log)
            targets_original = torch.expm1(targets_log)
            
            # For first 3 columns, only compute R² on non-zero targets
            # For last 2 columns, use all samples
            mask = torch.ones_like(targets_original, device=device)
            mask[:, :3] = is_nonzero  # Mask zeros for first 3 columns
            
            # Compute squared residuals, sums, and valid sample counts
            residuals_sq = ((pred_original - targets_original) ** 2) * mask
            ss_res += residuals_sq.sum(dim=0)
            sum_target += (targets_original * mask).sum(dim=0)
            sum_target_sq += ((targets_original ** 2) * mask).sum(dim=0)
            valid_samples += mask.sum(dim=0)

    # Compute R² per column, then weight
    r2_per_output = torch.zeros(5, device=device)
    for col in range(5):
        if valid_samples[col] > 0:
            mean_target = sum_target[col] / valid_samples[col]
            ss_tot = sum_target_sq[col] - valid_samples[col] * (mean_target ** 2)
            r2_per_output[col] = 1 - ss_res[col] / (ss_tot + 1e-10)
    
    # Weight the R² scores
    weights = torch.tensor(GIVEN_WEIGHTS, device=device)
    r2_weighted = (r2_per_output * weights).sum() / weights.sum()
    
    return total_loss / len(val_loader), r2_weighted

def compute_fold_metrics(epoch_train_losses, epoch_val_losses, epoch_val_r2s):
    best_val_idx = np.argmin(epoch_val_losses)
    best_val_loss = epoch_val_losses[best_val_idx]
    best_val_r2 = epoch_val_r2s[best_val_idx]
    overfit_metric = epoch_train_losses[best_val_idx] - best_val_loss
    stability_val_loss = np.mean(epoch_val_losses[-5:])
    stability_val_r2 = np.mean(epoch_val_r2s[-5:])

    metrics = {
        "best_val_loss": best_val_loss,
        "best_val_r2": best_val_r2,
        "overfit_metric": overfit_metric,
        "stability_val_loss": stability_val_loss,
        "stability_val_r2": stability_val_r2
    }
    return metrics

def print_fold_metrics(fold, metrics):
    print(f"\n--- Fold {fold + 1} Metrics Summary ---")
    print(f"Best Val Loss: {metrics['best_val_loss']:.4f}")
    print(f"Best Val R²: {metrics['best_val_r2']:.4f}")
    print(f"Overfit Metric (train - val at best epoch): {metrics['overfit_metric']:.4f}")
    print(f"Average Val Loss (last 5 epochs): {metrics['stability_val_loss']:.4f}")
    print(f"Average Val R² (last 5 epochs): {metrics['stability_val_r2']:.4f}")


def finetune_phase(pretrained_backbone=None, num_epochs=NUM_FT_EPOCHS):
    print("\n" + "=" * 50)
    print("PHASE 2: FINE-TUNING ON COMPETITION DATA")
    
    r2 = []
    fold_models = []

    for fold, (train_idx, val_idx) in enumerate(splits):
        print(f"\n--- Fold {fold + 1} Metrics Summary ---")
        train_df = dataset_df.iloc[train_idx].copy()
        val_df = dataset_df.iloc[val_idx].copy()
        train_loader, val_loader, train_dataset, val_dataset = create_data_loaders(train_df, val_df)

        model = FinetuneModel().to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

        
        epoch_train_losses = []
        epoch_val_losses = []
        epoch_val_r2s = []
    

        for epoch in range(num_epochs):
            epoch_start = time.time()
            train_loss = train_epoch(model, train_loader, optimizer)
            val_loss, val_r2 = validate_epoch(model, val_loader, val_dataset)

            epoch_train_losses.append(train_loss)
            epoch_val_losses.append(val_loss)
            epoch_val_r2s.append(val_r2.item())
            
            print_result(train_loss, val_loss, epoch_start, epoch, num_epochs, val_r2)

        r2.append(val_r2.item())
        fold_models.append(model)
        fold_metrics = compute_fold_metrics(epoch_train_losses, epoch_val_losses, epoch_val_r2s)
        print_fold_metrics(fold, fold_metrics)

    overall_r2 = np.array(r2).mean()
    
    print(f'\nOverall R² across all folds: {overall_r2:.4f}')

    return fold_models

In [315]:
# Create a small dataset instance
ds = BiomassDataset(dataset_df.head(12), base_path, transform=None)

# Loop over a few samples
for i in range(len(ds)):
    image, (is_nonzero, targets) = ds[i]
    print(f"Sample {i}:")
    print("is_nonzero:", is_nonzero)
    print("targets (masked & normalized):", targets)
    print("-" * 30)

Sample 0:
is_nonzero: tensor([0., 1., 1.])
targets (masked & normalized): tensor([ 0.0000,  1.3179,  0.5166,  0.7693, -0.2247])
------------------------------
Sample 1:
is_nonzero: tensor([0., 0., 1.])
targets (masked & normalized): tensor([ 0.0000,  0.0000, -0.0662, -1.7963, -1.3329])
------------------------------
Sample 2:
is_nonzero: tensor([1., 0., 0.])
targets (masked & normalized): tensor([ 0.4677,  0.0000,  0.0000, -2.0884, -1.6486])
------------------------------
Sample 3:
is_nonzero: tensor([0., 1., 1.])
targets (masked & normalized): tensor([0.0000, 1.2927, 0.8333, 0.9628, 0.3776])
------------------------------
Sample 4:
is_nonzero: tensor([1., 1., 1.])
targets (masked & normalized): tensor([-0.6543,  1.0714,  0.1785,  0.2743, -0.8088])
------------------------------
Sample 5:
is_nonzero: tensor([1., 1., 1.])
targets (masked & normalized): tensor([ 1.3331, -0.4460,  1.0623,  1.0311,  1.6514])
------------------------------
Sample 6:
is_nonzero: tensor([1., 1., 1.])
targets 

In [319]:
### NUM_FOLDS
# NUM_FT_EPOCHS
# kfold = GroupKFold(n_splits=NUM_FOLDS)
# groups = dataset_df['Month']
# splits = kfold.split(dataset_df, groups=groups)
from sklearn.model_selection import train_test_split
import numpy as np

# Make a single random split
train_idx, val_idx = train_test_split(
    np.arange(len(dataset_df)),
    test_size=0.2,
    shuffle=False,
    random_state=42
)

# Wrap it in a list so enumerate() still works
splits = [(train_idx, val_idx)]

final_model = finetune_phase(pretrained_backbone=False,num_epochs=2)
# --- efficientb 
# Epoch 1/40 - Train Loss: 0.1669, Val Loss: 0.1252 | R²: 0.2158 | Time: 0m 50s
# Epoch 2/40 - Train Loss: 0.1363, Val Loss: 0.1012 | R²: 0.3694 | Time: 0m 50s
# Epoch 3/40 - Train Loss: 0.1186, Val Loss: 0.0877 | R²: 0.4542 | Time: 0m 50s
# Epoch 4/40 - Train Loss: 0.1043, Val Loss: 0.0841 | R²: 0.4783 | Time: 0m 50s
# Epoch 5/40 - Train Loss: 0.0916, Val Loss: 0.0814 | R²: 0.4980 | Time: 0m 50s
# Epoch 6/40 - Train Loss: 0.0825, Val Loss: 0.0801 | R²: 0.5055 | Time: 0m 49s
# Epoch 7/40 - Train Loss: 0.0831, Val Loss: 0.0750 | R²: 0.5377 | Time: 0m 50s
# Epoch 8/40 - Train Loss: 0.0733, Val Loss: 0.0757 | R²: 0.5345 | Time: 0m 50s
# Epoch 9/40 - Train Loss: 0.0717, Val Loss: 0.0778 | R²: 0.5194 | Time: 0m 50s
# Epoch 10/40 - Train Loss: 0.0693, Val Loss: 0.0745 | R²: 0.5398 | Time: 0m 50s
# Epoch 11/40 - Train Loss: 0.0671, Val Loss: 0.0713 | R²: 0.5599 | Time: 0m 50s
# Epoch 12/40 - Train Loss: 0.0630, Val Loss: 0.0724 | R²: 0.5511 | Time: 0m 50s
# Epoch 13/40 - Train Loss: 0.0587, Val Loss: 0.0759 | R²: 0.5284 | Time: 0m 50s
# Epoch 14/40 - Train Loss: 0.0631, Val Loss: 0.0732 | R²: 0.5463 | Time: 0m 50s
# Epoch 15/40 - Train Loss: 0.0568, Val Loss: 0.0688 | R²: 0.5731 | Time: 0m 50s
# Epoch 16/40 - Train Loss: 0.0526, Val Loss: 0.0679 | R²: 0.5778 | Time: 0m 50s
# Epoch 17/40 - Train Loss: 0.0543, Val Loss: 0.0669 | R²: 0.5858 | Time: 0m 50s
# --- With  with higher weight decay



PHASE 2: FINE-TUNING ON COMPETITION DATA

--- Fold 1 Metrics Summary ---
Epoch 1/2 - Train Loss: 0.8684, Val Loss: 0.8227 | R²: -0.1976 | Time: 0m 31s
Epoch 2/2 - Train Loss: 0.7949, Val Loss: 0.7429 | R²: -0.1904 | Time: 0m 31s

--- Fold 1 Metrics Summary ---
Best Val Loss: 0.7429
Best Val R²: -0.1904
Overfit Metric (train - val at best epoch): 0.0521
Average Val Loss (last 5 epochs): 0.7828
Average Val R² (last 5 epochs): -0.1940

Overall R² across all folds: -0.1904


In [320]:
test_dataset = BiomassDataset(test_df, base_path, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
fold_models = final_model
ensemble_outputs = []
with torch.no_grad():
    for image, img_path in test_loader:
        image = image.to(device)
        
        # Get predictions from all models
        all_outputs = []
        for model in fold_models:
            model.eval()
            outputs = model(image)
            print(outputs)
            all_outputs.append(outputs)
            ensemble_outputs.append(torch.stack(all_outputs).mean(dim=0))
        
ensemble_outputs = torch.cat(ensemble_outputs, dim=0)


(tensor([[0.4832, 0.6633, 0.6894],
        [0.4832, 0.6633, 0.6894],
        [0.4832, 0.6633, 0.6894],
        [0.4832, 0.6633, 0.6894],
        [0.4832, 0.6633, 0.6894]], device='cuda:0'), tensor([[ 0.2535,  0.3230, -0.2218, -0.2500, -0.0024],
        [ 0.2535,  0.3230, -0.2218, -0.2500, -0.0024],
        [ 0.2535,  0.3230, -0.2218, -0.2500, -0.0024],
        [ 0.2535,  0.3230, -0.2218, -0.2500, -0.0024],
        [ 0.2535,  0.3230, -0.2218, -0.2500, -0.0024]], device='cuda:0'))


TypeError: expected Tensor as element 0 in argument 0, but got tuple

In [321]:
test_dataset = BiomassDataset(test_df, base_path, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Dictionary to store predictions grouped by image_path
preds_by_image = defaultdict(list)

with torch.no_grad():
    for image, img_paths in test_loader:
        image = image.to(device)
        
        # Get ensemble predictions from all models
        all_outputs = []
        for model in fold_models:
            model.eval()
            outputs = model(image)
            all_outputs.append(outputs)
        
        # Average across models (ensemble)
        # ensemble_batch = torch.stack(all_outputs).mean(dim=0)  # [batch_size, 5]
        # Collect regression and binary outputs separately
        all_regression = [out[1] for out in all_outputs]  # regression head
        all_binary = [out[0] for out in all_outputs]      # binary head
        
        # Ensemble (average) across models
        ensemble_regression = torch.stack(all_regression).mean(dim=0)  # [batch_size, 5]
        ensemble_binary = torch.stack(all_binary).mean(dim=0)          # [batch_size, 3]
        
        # Create final predictions
        final_preds = ensemble_regression.clone()
        
        # Apply hurdle logic for first 3 columns
        final_preds[:, :3] *= (ensemble_binary > 0.5).float()  # zero if binary predicts 0
        
        # Last 2 columns remain as regression outputs
        # Group predictions by image_path
        # Since BiomassDataset returns 2 items per image (left/right halves),
        # img_paths is a tuple/list of image paths (may have duplicates)
        # We group by image_path and will average left/right later
        batch_size = ensemble_batch.shape[0]
        for i in range(batch_size):
            img_path = img_paths[i]  # Get the image_path for this batch item
            preds_by_image[img_path].append(ensemble_batch[i].cpu())

# Average left/right predictions for each image
image_predictions = {}
for img_path, preds in preds_by_image.items():
    # Stack and average: [num_halves, 5] -> [5]
    # Each image should have exactly 2 predictions (left + right)
    stacked_preds = torch.stack(preds)  # [2, 5] for left and right halves
    image_predictions[img_path] = stacked_preds.mean(dim=0)  # Average to [5]ee

# Check predictions for first image
if len(image_predictions) > 0:
    first_img_path = list(image_predictions.keys())[0]
    print(f"Image: {first_img_path}")
    print(f"Predictions: {image_predictions[first_img_path].tolist()}")
    print(f"Target columns: {TARGET_COLS}")
else:
    print("No predictions available yet. Run the test prediction cell first.")

TypeError: expected Tensor as element 0 in argument 0, but got tuple

In [ ]:
# Denormalize all predictions back to original scale
for img_path in image_predictions:
    image_predictions[img_path] = denormalize_targets(image_predictions[img_path])

print(f"Denormalized predictions for {len(image_predictions)} images")


In [ ]:
ensemble_outputs[0].tolist()

In [ ]:
submission_df = pd.read_csv(submission_path)

for i, row in submission_df.iterrows():
    img_path = test_df[test_df['sample_id'] == row['sample_id']]['image_path'].values[0]
    target_name = test_df[test_df['sample_id'] == row['sample_id']]['target_name'].values[0]
    
    preds = ensemble_outputs
    target_idx = TARGET_COLS.index(target_name)
    submission_df.at[i, 'target'] = ensemble_outputs[i, target_idx].item()

submission_df.to_csv('submission.csv', index=False)
submission_df